In [ ]:
import tensorflow as tf
import tensorflow.keras.models
tf.keras.backend.clear_session()

In [ ]:
from flask import Flask, render_template, Response
from camera import Camera
import cv2
import numpy as np

In [ ]:
def load_model():
    new_mod=tf.keras.models.load_model(r'C:\Users\Hargun\Desktop\cancguip2\skinmodellatest2')
    new_mod._make_predict_function() 
#     print('model loaded') # just to keep track in your server
    return new_mod

In [ ]:
def modelresults(img):
    x=cv2.resize(img,(100,100))
    x=x.reshape(1,100,100,3)
    return x

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

def gen():
    vc = cv2.VideoCapture(0)
    color = (150, 200, 180)
    thickness = 2
    font = cv2.FONT_HERSHEY_SIMPLEX 
    while True:
        texval=''
        rval, frame = vc.read()
        zx=frame.shape
        res=modelresults(frame)
        mod=load_model()
        finresult=mod.predict(res)[0][0]
        if(finresult>0.5):
            texval='Malignant'
        else:
            texval='Benign'
        frame=cv2.putText(frame,texval, (int(zx[1]*(3/4)),int(zx[0]/4)), font,1, color, thickness, cv2.LINE_AA)
        frame=cv2.rectangle(frame,(int(zx[1]/4),int(zx[0]/4)),(int(zx[1]*(3/4)),int(zx[0]*(3/4))),(255,0,0),2)
        flag, encodedImage = cv2.imencode(".jpg",frame)
        yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + bytearray(encodedImage) + b'\r\n')
        tf.keras.backend.clear_session()

       


@app.route('/video_feed')
def video_feed():
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/video_feed_static')
def video_feed_static():
    return render_template('Videoplayingtemp.html')


app.run(host='0.0.0.0')

